In [ ]:
# Setup
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Lesson 7: Closed-Form Copula Families

This notebook presents the most commonly used copula families with closed-form expressions. We will examine elliptical copulas (Gaussian, Student-t) and Archimedean copulas (Clayton, Gumbel, Frank), exploring their mathematical formulations and sampling methods.

## Learning Objectives
- Understand the Gaussian and Student-t copulas
- Learn the generator function approach for Archimedean copulas
- Compare different copula families and their tail dependence properties

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
from IPython import display
import seaborn as sns
sns.set_theme(style="whitegrid")

import numpy as np
from scipy.stats import norm, \
    beta, cauchy, expon, rayleigh, uniform, multivariate_t, t
import pandas as pd
from scipy.optimize import brentq

## Elliptical Copulas

Elliptical copulas are derived from elliptical distributions. They are symmetric and characterized by a correlation matrix. The two most common elliptical copulas are the Gaussian and Student-t copulas.

### Gaussian Copula

The Gaussian copula is the dependence structure of a multivariate normal distribution. It is parameterized by a correlation matrix $R$.

**Definition:**

$$ C_R(u,v) = \Phi_R(\Phi^{-1}(u), \Phi^{-1}(v)) $$

where:
- $\Phi_R$ is the bivariate standard normal CDF with correlation matrix $R$
- $\Phi^{-1}$ is the inverse of the univariate standard normal CDF

**Key Properties:**
- Symmetric dependence (upper and lower tails behave identically)
- No tail dependence (asymptotically independent in both tails)
- Simple parameter interpretation ($\rho$ is the correlation coefficient)

In [ ]:
# Generate samples from a Gaussian copula
# Reference: https://documentation.sas.com/doc/en/etsug/15.2/etsug_copula_details03.htm

# Set correlation parameter
r = 0.8

# Define correlation matrix
P = np.asarray([
    [1, r],
    [r, 1]
])
d = P.shape[0]  # Dimension
n = 500         # Number of samples

# Sampling method: Cholesky decomposition
# 1. Generate independent standard normal samples
# 2. Transform using Cholesky factor to induce correlation
# 3. Apply normal CDF to get uniform margins (copula samples)
A = np.linalg.cholesky(P)
Z = np.random.normal(size=(n, d))
U_Gauss = norm.cdf(np.matmul(Z, A))

In [ ]:
# Visualize Gaussian copula with different marginal distributions
# Same dependence structure, different joint appearances

# Transform to Normal marginals
H1 = np.empty_like(U_Gauss)
H1[:, 0] = norm.ppf(U_Gauss[:, 0])
H1[:, 1] = norm.ppf(U_Gauss[:, 1])

plt.figure(figsize=(9, 3))

plt.subplot(1, 3, 1)
plt.scatter(H1[:, 0], H1[:, 1], alpha=0.2)
plt.xlabel('$\\mathcal{N}(0,1)$')
plt.ylabel('$\\mathcal{N}(0,1)$')

# Transform to Beta and Cauchy marginals
H2 = np.empty_like(U_Gauss)
H2[:, 0] = beta.ppf(U_Gauss[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_Gauss[:, 1])

plt.subplot(1, 3, 2)
plt.scatter(H2[:, 0], H2[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Beta}(2,5)$')
plt.ylabel('$\\mathrm{Cauchy}(0,1)$')

# Transform to Exponential marginals
H3 = np.empty_like(U_Gauss)
H3[:, 0] = expon.ppf(U_Gauss[:, 0])
H3[:, 1] = expon.ppf(U_Gauss[:, 1])

plt.subplot(1, 3, 3)
plt.scatter(H3[:, 0], H3[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Exponential}(1)$')
plt.ylabel('$\\mathrm{Exponential}(1)$')

plt.suptitle('Gaussian Copula $\\rho=%0.02f$' % (P[0, 1],))
plt.tight_layout()

### Student-t Copula

The Student-t copula is the dependence structure of a multivariate t-distribution. It has an additional parameter $\nu$ (degrees of freedom) compared to the Gaussian copula.

**Definition:**

$$ C_{\Sigma,\nu}(u,v) = \mathbf{t}_{\Sigma,\nu}(t_\nu^{-1}(u), t_\nu^{-1}(v)) $$

where:
- $\mathbf{t}_{\Sigma,\nu}$ is the bivariate t-distribution CDF with correlation matrix $\Sigma$ and $\nu$ degrees of freedom
- $t_\nu^{-1}$ is the inverse of the univariate t-distribution CDF with $\nu$ degrees of freedom

**Key Properties:**
- Symmetric dependence
- Has tail dependence (stronger dependence in the tails than Gaussian)
- Approaches Gaussian copula as $\nu \rightarrow \infty$

In [ ]:
# Generate samples from a Student-t copula
# Reference: https://documentation.sas.com/doc/en/etsug/15.2/etsug_copula_details06.htm

# Set parameters
r = 0.8                                      # Correlation parameter
mu = np.asarray([0, 0])                      # Mean vector
sigma = np.asarray([[1, r], [r, 1]])         # Shape matrix
nu = 5                                       # Degrees of freedom (lower = heavier tails)

# Sampling method:
# 1. Generate multivariate t samples
# 2. Apply univariate t CDF to each margin
rv = multivariate_t(mu, sigma, df=nu)
X = rv.rvs(size=n)
U_StudentT = np.zeros_like(X)
t_dist = t(nu)
U_StudentT[:, 0] = t_dist.cdf(X[:, 0])
U_StudentT[:, 1] = t_dist.cdf(X[:, 1])

In [ ]:
# Visualize Student-t copula with different marginal distributions

# Transform to Normal marginals
H1 = np.empty_like(U_StudentT)
H1[:, 0] = norm.ppf(U_StudentT[:, 0])
H1[:, 1] = norm.ppf(U_StudentT[:, 1])

plt.figure(figsize=(9, 3))

plt.subplot(1, 3, 1)
plt.scatter(H1[:, 0], H1[:, 1], alpha=0.2)
plt.xlabel('$\\mathcal{N}(0,1)$')
plt.ylabel('$\\mathcal{N}(0,1)$')

# Transform to Beta and Cauchy marginals
H2 = np.empty_like(U_StudentT)
H2[:, 0] = beta.ppf(U_StudentT[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_StudentT[:, 1])

plt.subplot(1, 3, 2)
plt.scatter(H2[:, 0], H2[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Beta}(2,5)$')
plt.ylabel('$\\mathrm{Cauchy}(0,1)$')

# Transform to Exponential marginals
H3 = np.empty_like(U_StudentT)
H3[:, 0] = expon.ppf(U_StudentT[:, 0])
H3[:, 1] = expon.ppf(U_StudentT[:, 1])

plt.subplot(1, 3, 3)
plt.scatter(H3[:, 0], H3[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Exponential}(1)$')
plt.ylabel('$\\mathrm{Exponential}(1)$')

plt.suptitle('Student-t Copula $\\rho=%0.02f$, $\\nu=%d$' % (P[0, 1], nu))
plt.tight_layout()

## Archimedean Copulas

Archimedean copulas are defined using a generator function $\phi$. They offer flexibility and can capture asymmetric tail dependence.

**General Form:**

$$ C(u,v) = \phi^{[-1]}(\phi(u) + \phi(v)) $$

where $\phi$ is the generator function satisfying:
- $\phi: [0, 1] \rightarrow [0, \infty]$
- $\phi(1) = 0$
- $\phi$ is strictly decreasing and convex

The pseudo-inverse is defined as:
$$ \phi^{[-1]}(t) = 
\begin{cases}
\phi^{-1}(t), & 0 \leq t \leq \phi(0) \\
0, & \phi(0) \leq t < \infty
\end{cases}
$$

Different choices of $\phi$ give rise to different copula families.

### Clayton Copula

The Clayton copula exhibits strong lower tail dependence and no upper tail dependence. It is useful for modeling situations where extreme low values tend to occur together.

**Generator and Copula Function:**

$$ \phi(t) = \frac{1}{\alpha}(t^{-\alpha} - 1), \quad \alpha \in [-1, \infty) \setminus \{0\}$$
$$ \phi^{[-1]}(t) = (\alpha t + 1)^{-1/\alpha} $$
$$ C(u,v) = (u^{-\alpha} + v^{-\alpha} - 1)^{-1/\alpha} $$

**Parameter Interpretation:**
- $\alpha > 0$: positive dependence (higher $\alpha$ = stronger dependence)
- $\alpha \rightarrow 0$: independence
- $\alpha = -1$: countermonotonicity (Frechet-Hoeffding lower bound)

In [ ]:
# Generate samples from a Clayton copula
# Reference: https://medium.com/@financialnoob/introduction-to-copulas-part-2-9de74010ed87

# Clayton parameter (higher = stronger lower tail dependence)
alpha = 6

# Sampling method: Conditional distribution method
# 1. Generate u uniformly
# 2. Generate t uniformly 
# 3. Compute v using the inverse conditional distribution
u = np.random.rand(n)
t_rand = np.random.rand(n)
v = ((t_rand / u**(-alpha-1))**(-alpha/(1+alpha)) - u**(-alpha) + 1)**(-1/alpha)
U_clayton = np.vstack([u, v]).T

In [ ]:
# Visualize Clayton copula with different marginal distributions

# Transform to Normal marginals
H1 = np.empty_like(U_clayton)
H1[:, 0] = norm.ppf(U_clayton[:, 0])
H1[:, 1] = norm.ppf(U_clayton[:, 1])

plt.figure(figsize=(9, 3))

plt.subplot(1, 3, 1)
plt.scatter(H1[:, 0], H1[:, 1], alpha=0.2)
plt.xlabel('$\\mathcal{N}(0,1)$')
plt.ylabel('$\\mathcal{N}(0,1)$')

# Transform to Beta and Cauchy marginals
H2 = np.empty_like(U_clayton)
H2[:, 0] = beta.ppf(U_clayton[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_clayton[:, 1])

plt.subplot(1, 3, 2)
plt.scatter(H2[:, 0], H2[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Beta}(2,5)$')
plt.ylabel('$\\mathrm{Cauchy}(0,1)$')

# Transform to Exponential marginals
H3 = np.empty_like(U_clayton)
H3[:, 0] = expon.ppf(U_clayton[:, 0])
H3[:, 1] = expon.ppf(U_clayton[:, 1])

plt.subplot(1, 3, 3)
plt.scatter(H3[:, 0], H3[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Exponential}(1)$')
plt.ylabel('$\\mathrm{Exponential}(1)$')

plt.suptitle(r'Clayton Copula $\alpha=%0.02f$' % (alpha,))
plt.tight_layout()

### Gumbel Copula

The Gumbel copula exhibits strong upper tail dependence and no lower tail dependence. It is useful for modeling situations where extreme high values tend to occur together.

**Generator and Copula Function:**

$$ \phi(t) = (-\ln t)^\alpha, \quad \alpha \in [1, \infty) $$
$$ \phi^{-1}(t) = e^{-t^{1/\alpha}} $$
$$ C(u,v) = \exp\left(-\left((-\ln u)^\alpha + (-\ln v)^\alpha\right)^{1/\alpha}\right) $$

**Parameter Interpretation:**
- $\alpha = 1$: independence
- $\alpha \rightarrow \infty$: comonotonicity (Frechet-Hoeffding upper bound)

In [ ]:
# Gumbel copula generator function and inverse
def gumbel_phi(t, alpha):
    """Gumbel generator function"""
    return (-np.log(t))**alpha

def gumbel_phi_inv(t, alpha):
    """Inverse of Gumbel generator function"""
    return np.exp(-t**(1/alpha))

def gumbel_K(t, alpha):
    """Kendall distribution function for Gumbel copula"""
    return t * (alpha - np.log(t)) / alpha

# Generate Gumbel copula samples using the Marshall-Olkin method
alpha = 6  # Gumbel parameter (higher = stronger upper tail dependence)

t1 = np.random.rand(n)
t2 = np.random.rand(n)

# Solve for w using Brent's method
w = []
for t_val in t2:
    func = lambda w: gumbel_K(w, alpha=alpha) - t_val
    w.append(brentq(func, 0.0000000001, 0.9999999999))
w = np.array(w).flatten()

# Compute u and v
u = gumbel_phi_inv(t1 * gumbel_phi(w, alpha=alpha), alpha=alpha)
v = gumbel_phi_inv((1-t1) * gumbel_phi(w, alpha=alpha), alpha=alpha)
U_gumbel = np.vstack([u, v]).T

In [ ]:
# Visualize Gumbel copula with different marginal distributions

# Transform to Normal marginals
H1 = np.empty_like(U_gumbel)
H1[:, 0] = norm.ppf(U_gumbel[:, 0])
H1[:, 1] = norm.ppf(U_gumbel[:, 1])

plt.figure(figsize=(9, 3))

plt.subplot(1, 3, 1)
plt.scatter(H1[:, 0], H1[:, 1], alpha=0.2)
plt.xlabel('$\\mathcal{N}(0,1)$')
plt.ylabel('$\\mathcal{N}(0,1)$')

# Transform to Beta and Cauchy marginals
H2 = np.empty_like(U_gumbel)
H2[:, 0] = beta.ppf(U_gumbel[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_gumbel[:, 1])

plt.subplot(1, 3, 2)
plt.scatter(H2[:, 0], H2[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Beta}(2,5)$')
plt.ylabel('$\\mathrm{Cauchy}(0,1)$')

# Transform to Exponential marginals
H3 = np.empty_like(U_gumbel)
H3[:, 0] = expon.ppf(U_gumbel[:, 0])
H3[:, 1] = expon.ppf(U_gumbel[:, 1])

plt.subplot(1, 3, 3)
plt.scatter(H3[:, 0], H3[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Exponential}(1)$')
plt.ylabel('$\\mathrm{Exponential}(1)$')

plt.suptitle(r'Gumbel Copula $\alpha=%0.02f$' % (alpha,))
plt.tight_layout()

### Frank Copula

The Frank copula has no tail dependence in either tail. It is symmetric and can model both positive and negative dependence.

**Generator and Copula Function:**

$$ \phi(t) = -\ln\left( \frac{e^{-\alpha t}-1}{e^{-\alpha}-1} \right), \quad \alpha \in (-\infty, \infty) \setminus \{0\} $$ 
$$ \phi^{-1}(t) = -\frac{1}{\alpha}\ln\left( \frac{e^{-\alpha}-1}{e^t} + 1 \right) $$
$$ C(u,v) = -\frac{1}{\alpha} \ln \left( 1 + \frac{(e^{-\alpha u}-1)(e^{-\alpha v}-1)}{e^{-\alpha}-1} \right) $$

**Parameter Interpretation:**
- $\alpha > 0$: positive dependence
- $\alpha < 0$: negative dependence
- $\alpha \rightarrow 0$: independence

In [ ]:
# Frank copula generator function and inverse
def frank_phi(t, alpha):
    """Frank generator function"""
    return -np.log((np.exp(-alpha*t) - 1) / (np.exp(-alpha) - 1))

def frank_phi_inv(t, alpha):
    """Inverse of Frank generator function"""
    return -1/alpha * np.log((np.exp(-alpha) - 1) / np.exp(t) + 1)

def frank_K(t, alpha):
    """Kendall distribution function for Frank copula"""
    return (t + (1 - np.exp(alpha*t)) * np.log((1-np.exp(alpha*t)) * 
                                               np.exp(-alpha*t+alpha) / (1-np.exp(alpha))) / alpha)

# Generate Frank copula samples using the Marshall-Olkin method
alpha = 6  # Frank parameter (higher = stronger positive dependence)

t1 = np.random.rand(n)
t2 = np.random.rand(n)

# Solve for w using Brent's method
w = []
for t_val in t2:
    func = lambda w: frank_K(w, alpha=alpha) - t_val
    w.append(brentq(func, 0.0000000001, 0.9999999999))
w = np.array(w).flatten()

# Compute u and v
u = frank_phi_inv(t1 * frank_phi(w, alpha=alpha), alpha=alpha)
v = frank_phi_inv((1-t1) * frank_phi(w, alpha=alpha), alpha=alpha)
U_frank = np.vstack([u, v]).T

In [ ]:
# Visualize Frank copula with different marginal distributions

# Transform to Normal marginals
H1 = np.empty_like(U_frank)
H1[:, 0] = norm.ppf(U_frank[:, 0])
H1[:, 1] = norm.ppf(U_frank[:, 1])

plt.figure(figsize=(9, 3))

plt.subplot(1, 3, 1)
plt.scatter(H1[:, 0], H1[:, 1], alpha=0.2)
plt.xlabel('$\\mathcal{N}(0,1)$')
plt.ylabel('$\\mathcal{N}(0,1)$')

# Transform to Beta and Cauchy marginals
H2 = np.empty_like(U_frank)
H2[:, 0] = beta.ppf(U_frank[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_frank[:, 1])

plt.subplot(1, 3, 2)
plt.scatter(H2[:, 0], H2[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Beta}(2,5)$')
plt.ylabel('$\\mathrm{Cauchy}(0,1)$')

# Transform to Exponential marginals
H3 = np.empty_like(U_frank)
H3[:, 0] = expon.ppf(U_frank[:, 0])
H3[:, 1] = expon.ppf(U_frank[:, 1])

plt.subplot(1, 3, 3)
plt.scatter(H3[:, 0], H3[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Exponential}(1)$')
plt.ylabel('$\\mathrm{Exponential}(1)$')

plt.suptitle(r'Frank Copula $\alpha=%0.02f$' % (alpha,))
plt.tight_layout()

## Comparison of Copula Families

Let us compare all five copula families side by side to observe their different dependence structures. Notice:
- **Gaussian & Student-t**: Symmetric dependence patterns
- **Clayton**: Concentration in lower-left corner (lower tail dependence)
- **Gumbel**: Concentration in upper-right corner (upper tail dependence)
- **Frank**: Symmetric with no tail dependence

In [ ]:
# Compare all copula samples in the unit square
plt.figure(figsize=(9, 6))

plt.subplot(2, 3, 1)
plt.scatter(U_Gauss[:, 0], U_Gauss[:, 1], alpha=0.2)
plt.xlabel('U')
plt.ylabel('V')
plt.title('Gaussian Copula $\\rho=%0.02f$' % (P[0, 1],))

plt.subplot(2, 3, 2)
plt.scatter(U_StudentT[:, 0], U_StudentT[:, 1], alpha=0.2)
plt.xlabel('U')
plt.ylabel('V')
plt.title('Student-t Copula $\\rho=%0.02f$, $\\nu=%d$' % (P[0, 1], nu))

plt.subplot(2, 3, 4)
plt.scatter(U_frank[:, 0], U_frank[:, 1], alpha=0.2)
plt.xlabel('U')
plt.ylabel('V')
plt.title(r'Frank Copula $\alpha=%0.02f$' % (alpha,))

plt.subplot(2, 3, 5)
plt.scatter(U_gumbel[:, 0], U_gumbel[:, 1], alpha=0.2)
plt.xlabel('U')
plt.ylabel('V')
plt.title(r'Gumbel Copula $\alpha=%0.02f$' % (alpha,))

plt.subplot(2, 3, 6)
plt.scatter(U_clayton[:, 0], U_clayton[:, 1], alpha=0.2)
plt.xlabel('U')
plt.ylabel('V')
plt.title(r'Clayton Copula $\alpha=%0.02f$' % (alpha,))

plt.tight_layout()